In [14]:
from openai import OpenAI
import pprint
import time
import ast
import re
client = OpenAI()
start_reading = time.time()
audio_file = open("memento-sample-audio.mp3", "rb")
end_reading = time.time()
print(f"Reading audio file took {end_reading - start_reading} seconds")

start_transcription = time.time()
transcript = client.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["segment"]
)
end_transcription = time.time()
print(f"Transcription took {end_transcription - start_transcription} seconds")

print(transcript)
pprint.pp(transcript)  # Assuming transcript is already in a print-friendly format


Reading audio file took 0.0009987354278564453 seconds
Transcription took 6.408037424087524 seconds
Transcription(text="Hey man, how's it going? Good, good. How about you? What's your favorite animal, man? Pretty good. My favorite animal is dinosaur. But let me start, like, what's your name? I'm Arush. What's yours? I'm Vig, by the way. Nice to meet you. So how do you like the conference? It's pretty cool. It's really big. I'm meeting a lot of really cool people. My favorite animal is the capybara, so I've been finding a lot of capybara lovers around here. It's been nice. Tell me more about capybara. They're just really goofy. Interesting. Is that why you like them? Yeah, yeah. It's... That's very interesting, man. Like, I've never seen someone who's a fan of capybara. It's the first time. Yeah, they're like huge rats, basically. Oh, okay. Interesting. They're really cute, and there's been a trend going on, going around on, like, Instagram and stuff of reels of capybaras doing random st

In [15]:

# get the text from the transcript
speaker = 0
cleaned_text = []
for dict in transcript.segments:
    print(dict['text'])
    if speaker == 0:
        cleaned_text.append(f"Speaker A: {dict['text']}")
        speaker = 1
    else:
        cleaned_text.append(f"Speaker B: {dict['text']}")
        speaker = 0


print(cleaned_text)

 Hey man, how's it going?
 Good, good. How about you? What's your favorite animal, man?
 Pretty good. My favorite animal is dinosaur. But let me start, like, what's your name?
 I'm Arush. What's yours?
 I'm Vig, by the way. Nice to meet you. So how do you like the conference?
 It's pretty cool. It's really big. I'm meeting a lot of really cool people.
 My favorite animal is the capybara, so I've been finding a lot of capybara lovers around here. It's been nice.
 Tell me more about capybara.
 They're just really goofy.
 Interesting. Is that why you like them?
 Yeah, yeah. It's...
 That's very interesting, man. Like, I've never seen someone who's a fan of capybara.
 It's the first time.
 Yeah, they're like huge rats, basically.
 Oh, okay. Interesting.
 They're really cute, and there's been a trend going on, going around on, like, Instagram and stuff of
 reels of capybaras doing random stuff. They're like pandas, by the way.
 That's amazing. It kind of reminds me of how, you know, people 

In [19]:
formatted_text = " \n\n".join(cleaned_text)
print(formatted_text)

Speaker A:  Hey man, how's it going? 

Speaker B:  Good, good. How about you? What's your favorite animal, man? 

Speaker A:  Pretty good. My favorite animal is dinosaur. But let me start, like, what's your name? 

Speaker B:  I'm Arush. What's yours? 

Speaker A:  I'm Vig, by the way. Nice to meet you. So how do you like the conference? 

Speaker B:  It's pretty cool. It's really big. I'm meeting a lot of really cool people. 

Speaker A:  My favorite animal is the capybara, so I've been finding a lot of capybara lovers around here. It's been nice. 

Speaker B:  Tell me more about capybara. 

Speaker A:  They're just really goofy. 

Speaker B:  Interesting. Is that why you like them? 

Speaker A:  Yeah, yeah. It's... 

Speaker B:  That's very interesting, man. Like, I've never seen someone who's a fan of capybara. 

Speaker A:  It's the first time. 

Speaker B:  Yeah, they're like huge rats, basically. 

Speaker A:  Oh, okay. Interesting. 

Speaker B:  They're really cute, and there's 

In [40]:
import anthropic



client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
)
message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=500,
    temperature=0,
    system="You are an expert chat identifier who is attached to a set of smart glasses the user is wearing. Your task is to identify whether this conversation is an introduction to the user or not. You will be given the conversation within <conversation> XML tags. If so, you will respond with 'Y' enclosed in <ans> tags. If not, you will respond with 'N' enclosed in <ans> tags. Before you respond, analyze the conversation carefully within <thinking> tags. Take a deep breath and think step by step. Do not preface your thinking tags with anything, and be concise.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"<ans>{formatted_text}</ans>"
                }
            ]
        }
    ],
    stop_sequences=["</ans>"]
)
print(message.content[0].text[-1])


if str(message.content[0].text[-1]) == "Y":
    print("This is an introduction")
elif str(message.content[0].text[-1])[-1] == "N":
    print("This is not an introduction")
else:
    print("Something went wrong")

Y
This is an introduction
